In [ ]:
import sys
import os

# Ajouter le répertoire parent au PYTHONPATH
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.append(parent_dir)

# Importer les fonctions du module mongo_utils
from mongo_insertion import insert_data_from_json_to_mongodb

# Spécifiez le chemin de votre fichier JSON
json_file_path = r'C:\Users\MSI KATANA B13V\Downloads\de_satisfaction_client-develop\de_satisfaction_client-develop\df_commentaires_par_entreprise.json'  # Remplacez par le chemin réel de votre fichier JSON

# Utiliser la fonction pour insérer les données dans MongoDB
insert_data_from_json_to_mongodb(json_file_path)


In [ ]:

import sys
import os

# Ajouter le répertoire parent 'src' au PYTHONPATH
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.append(parent_dir)

from sql_lite_insertion import insert_companies_and_reviews

# Exemple d'utilisation de la fonction
import pandas as pd

# Charger votre DataFrame (df) depuis un fichier ou une autre source
df = pd.read_csv(r"C:\Users\MSI KATANA B13V\Downloads\de_satisfaction_client-develop\de_satisfaction_client-develop\data\informations_entreprises.csv", sep=',')  # Remplacez par le chemin de votre fichier CSV

# Appeler la fonction pour insérer les données
insert_companies_and_reviews(df)


In [4]:

%load_ext sql
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 50
%sql sqlite:///../src/trustscore.db
%reload_ext sql



In [7]:
from sklearn.impute import SimpleImputer
import numpy as np

def company_preprocessing(df):
    """
    Prétraite les données des entreprises avant l'insertion.
    
    Paramètres:
    - df (DataFrame): Le DataFrame contenant les données brutes.

    Retourne:
    - df (DataFrame): Le DataFrame nettoyé et prétraité.
    """
    # Renommer les colonnes
    df.rename(columns={"review": "nombre_reviews"}, inplace=True)

    # Supprimer les espaces blancs et normaliser les valeurs
    df['town'] = df['town'].astype(str).str.strip().str.upper().replace('nan', 'NaN')
    df['country'] = df['country'].astype(str).str.strip().replace('nan', 'NaN')
    df['institution_type'] = df['institution_type'].astype(str).str.strip().replace('nan', 'NaN')

    # Remplacer les valeurs NaN avec les valeurs les plus fréquentes
    imputer = SimpleImputer(missing_values='NaN', strategy='most_frequent')
    df[['town', 'country', 'institution_type']] = imputer.fit_transform(df[['town', 'country', 'institution_type']])

    # Convertir les colonnes numériques et gérer les valeurs NaN
    df['trust_score'] = pd.to_numeric(df['trust_score'], errors='coerce').fillna(0)
    df['five_star_%'] = df['five_star_%'].astype(str).replace('nan', '0').str.rstrip('%').astype(float) / 100
    df['nombre_reviews'] = pd.to_numeric(df['nombre_reviews'], errors='coerce').fillna(0)

    # Raccourcir le nom des entreprises
    df['short_company_name'] = df['company_name']
    df['company_name'] = df['short_company_name'].apply(lambda x: str(x).strip()[:7] + "...")

    return df


In [ ]:
# Charger votre DataFrame (df) depuis un fichier ou une autre source
df = pd.read_csv(r"C:\Users\MSI KATANA B13V\Downloads\de_satisfaction_client-develop\de_satisfaction_client-develop\data\informations_entreprises.csv", sep=',')  

company_preprocessing(df)

In [ ]:
%%sql
CREATE TABLE companies_infos (
  id INTEGER NOT NULL,
  town VARCHAR(50), 
  country VARCHAR(50),
  institution_type VARCHAR(100),
  company_name VARCHAR(100),
  trust_score  FLOAT, 
  nombre_reviews INTEGER,
  five_star_percentage REAL,
  short_company_name VARCHAR(50),
  PRIMARY KEY (id));

In [1]:
import sqlite3
import pandas as pd

def insert_companies_and_reviews(df, db_path='../src/trustscore.db'):
    """
    Insère les données d'entreprises et de reviews dans les tables appropriées.

    Paramètres:
    - df (DataFrame): Le DataFrame prétraité contenant les données des entreprises et des reviews.
    - db_path (str): Le chemin de la base de données SQLite.

    Retourne:
    - None
    """
    try:
        # Connexion à la base de données SQLite
        conn = sqlite3.connect(db_path)

        # Convertir la colonne en string avant manipulation
        if 'five_star_%' in df.columns:
            # Conversion sécurisée en chaîne de caractères
            df['five_star_%'] = df['five_star_%'].astype(str)  # Convertir en chaîne pour éviter l'erreur .str

            # Manipuler les valeurs pour enlever le % et diviser par 100
            df['five_star_percentage'] = df['five_star_%'].str.rstrip('%').replace('nan', '0').astype(float) / 100

            # Supprimer la colonne originale 'five_star_%' après conversion
            df.drop(columns=['five_star_%'], inplace=True)

        # Vérifier le DataFrame avant insertion
        print("Données avant insertion :")
        print(df.head())
        print(df.dtypes)

        # Insertion des données dans la table `companies_infos`
        df.to_sql('companies_infos', conn, if_exists='append', index=False)

        print("Insertion des données terminée avec succès.")

    except Exception as e:
        print(f"Erreur lors de l'insertion des données : {e}")

    finally:
        # Fermer la connexion
        conn.close()


In [14]:
import sqlite3

def truncate_table(db_path='../src/trustscore.db', table_name='companies_infos'):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        # Supprime toutes les lignes de la table
        cursor.execute(f"DELETE FROM {table_name};")
        # Réinitialise l'auto-incrémentation (utile si la table a une colonne ID auto-incrémentée)
        #cursor.execute(f"DELETE FROM sqlite_sequence WHERE name='{table_name}';")
        conn.commit()
        print(f"Table {table_name} vidée avec succès.")
    except Exception as e:
        print(f"Erreur lors de la vidange de la table : {e}")
    finally:
        conn.close()

# Appel de la fonction pour vider la table
truncate_table()


Table companies_infos vidée avec succès.


In [16]:
%%sql
select count(1)
from companies_infos

count(1)
24


In [8]:
import pandas as pd
import os


base_path =  '../data'
csv_file_path = os.path.join(base_path, 'informations_entreprises.csv')
json_file_path = os.path.join(base_path, 'df_commentaires_par_entreprise.json')

df = pd.read_csv(csv_file_path)

# df.head(5)
# Étape 2 : Preprocessing des données
print("Preprocessing des données...")
df = company_preprocessing(df)  # Appel de la fonction de preprocessing pour nettoyer et transformer les données

#df.head(5)

# Vérification des types de données après preprocessing
print("Types de données après preprocessing :")
print(df.dtypes)

        # Étape 3 : Insertion des données dans SQLite
#print("Insertion des données dans SQLite...")
insert_companies_and_reviews(df)




Preprocessing des données...
Types de données après preprocessing :
town                   object
country                object
institution_type       object
company_name           object
trust_score           float64
nombre_reviews          int64
five_star_%           float64
short_company_name     object
dtype: object
Données avant insertion :
       town        country institution_type company_name  trust_score  \
0  PORTLAND  United States     Credit Union   Evergre...          4.9   
1   LINCOLN  United States  Mortgage Lender   Liberty...          4.7   
2    IRVINE  United States              ATM   GoHenry...          4.3   
3   CHICAGO  United States              ATM   Crypto ...          4.1   
4   ATLANTA  United States             Bank   Anytime...          4.5   

   nombre_reviews          short_company_name  five_star_percentage  
0             343      Evergreen Credit Union                0.0096  
1             308  Liberty First Credit Union                0.0092  
2  

In [ ]:
import sqlite3
import pandas as pd

def insert_companies_and_reviews(df, db_path='trustscore.db'):
    """
    Insère les données d'entreprises et de reviews dans les tables appropriées.

    Paramètres:
    - df (DataFrame): Le DataFrame prétraité contenant les données des entreprises et des reviews.
    - db_path (str): Le chemin de la base de données SQLite.

    Retourne:
    - None
    """
    try:
        # Connexion à la base de données SQLite
        conn = sqlite3.connect(db_path)

        # Vérification et conversion de la colonne 'five_star_%' en string si nécessaire
        if 'five_star_%' in df.columns:
            df['five_star_%'] = df['five_star_%'].astype(str)  # Convertir en string pour éviter les erreurs
            # Supprimer le symbole % et convertir en float, puis diviser par 100
            df['five_star_percentage'] = df['five_star_%'].str.rstrip('%').replace('nan', '0').astype(float) / 100
            df.drop(columns=['five_star_%'], inplace=True)  # Supprimer la colonne originale après conversion
        
        # Vérifier le DataFrame avant insertion
        print("Données avant insertion :")
        print(df.head())

        # Insertion des données dans la table `companies_infos`
        df.to_sql('companies_infos', conn, if_exists='append', index=False)

        print("Insertion des données terminée avec succès.")

    except Exception as e:
        print(f"Erreur lors de l'insertion des données : {e}")

    finally:
        # Fermer la connexion
        conn.close()


In [ ]:
insert_companies_and_reviews(df)